# Project:Telephone station simulation
### Stu. name: Amirali Khatib
### Prof. name: Dr.Mohammad Ranjbar

**Problem:**

There is a calling station in police office. 2 types of telephone call happen in the station: **Emergency call** and **Ordinary call**.
A new applicant to make ordinary call enter the system every $t_{1} = Uniform(4,6)$ minutes. and each $t_{2} = Uniform(10,20)$ minutes person wants to make an emergency call. each on these types of calls has their own queue. In addition, not only do the emergency calls have a higher priority than ordinary call, but they also should stop the probable in progress personal call. After the completion of emergency call the incompleted ordinary call could continue the call.
The time duration for Emergency call and the Personal call are relatively $Uniform(2,4)$ and $Uniform(0,4)$. **20 percent** of people who complete their personal call want to make another personal call and they should be given the lowest priority at that time.

what we want to do is to simulate the telephone station system until at least 200 calls for each type (emergency and ordinary). and find the busy time proportion for telephone station.

**Status variable:**
* **CE(t)** ~ being busy(1) or free(0) for emergency call at time "t"
* **CO(t)** ~ being busy(1) or free(0) for ordinary call at time "t"
* **CO2(t)** ~ being busy(1) or free(0) for 2nd ordinary call at time "t"
* **CS(t)** ~ being busy(1) or free(0) for incompleted call at time "t"
* **QE(t)** ~ queue status for emergency calls at time "t"
* **QO(t)** ~ queue status for ordinary calls at time "t" 
* **QO2(t)** ~ queue status for 2nd ordinary calls at time "t"
* **QS(t)** ~ queu status for incompleted calls at time "t"

In [43]:
import numpy as np

In [1]:
CE = 0  # Emergecy call status (0,1)
CO = 0  # Ordinary call status (0,1)
CO2 = 0 # 2nd ordinary call status (0,1) 
CS = 0  # Unfinished call status (0,1)
QE = 0  # Emergency call queue status
QO = 0  # Ordinary call queue status
QO2 = 0 # 2nd ordinary call queue status
QS = 0  # Unfinished call queue status

In [45]:
n_sim_calls = int(input('Enter number of calls: '))
current_time = 0
FEL = []
CS_event_type = None
busy_time = 0
start_time = 0
n = 0

Enter number of calls: 200


In [46]:
FEL.append((current_time, 'Ao'))
FEL.append((current_time, 'Ae'))
FEL.sort()

In [47]:
def telephone_status():
    global CE, CO, CO2, CS
    return CE+CO+CO2+CS

In [48]:
def sudden_emergence():
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    
    CE = 1
    QS += 1
    service_time = float(np.random.uniform(10,20,1))
    for i in range(len(FEL)):
        if (FEL[i][1] == 'Do') | (FEL[i][1] == 'Do2'):
            CS_event_type = FEL[i][1]
            remaining_time = FEL[i][0] - current_time
            FEL.pop(i)
            
    FEL.append((current_time+service_time, 'De'))  
    FEL.sort()
    return None

In [49]:
def arrival_o(event_type):
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    FEL.append((current_time+float(np.random.uniform(4,6,1)), 'Ao'))
    FEL.sort()
    
    if event_type == 'Ao':
        if telephone_status() == 0:
            start_time = current_time
            CO = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do'))
            FEL.sort()
        elif telephone_status() == 1:
            QO += 1    
    
    elif event_type == 'Ao2':
        if telephone_status() == 1:
            QO2 += 1
        elif telephone_status() == 0:
            start_time = current_time
            CO2 = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do2'))
            FEL.sort()
        
    return None

In [50]:
def arrival_e():
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    FEL.append((current_time+float(np.random.uniform(10,20,1)), 'Ae'))
    FEL.sort()
    
    if telephone_status() == 0:
        start_time = current_time
        CE = 1
        service_time = float(np.random.uniform(2,4,1))
        FEL.append((current_time + service_time, 'De'))
        FEL.sort()
    elif telephone_status() == 1:
        if CE == 1:
            QE += 1
        elif CE == 0:
            sudden_emergence()
            if CS == 1:
                CS = 0    
            elif CO == 1:
                CO = 0
            elif CO2 == 1:
                CO2 = 0
    
    return None 

In [51]:
def arrival(event_type):
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    
    if (event_type == 'Ao') & (event_type == 'Ao2'):
        arrival_o()

    elif event_type == 'Ae':
        arrival_e()
    
    return None  

In [52]:
def departure_o(event_type):
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    if event_type == 'Do2':
        CO2 = 0
        if QO > 0:
            QO -= 1
            CO = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do'))
            FEL.sort()
        
        elif QO2 > 0:
            QO2 -= 1
            CO2 = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time,'Do2'))
            FEL.sort()
        
        else:
            busy_time += (current_time - start_time)
            

    elif event_type == 'Do':
        CO = 0
        if QO > 0:
            QO -= 1
            CO = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do'))
            FEL.sort()
            
        elif QO2 > 0:
            QO2 -= 1
            CO2 = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do2'))
            FEL.sort()
            
        else:
            busy_time += (current_time - start_time)


        p = float(np.random.uniform(0,1,1))
        if p <= 0.2:
            FEL.append((current_time,'Ao2'))
            FEL.sort()
        else:
            pass

    return None

In [53]:
def departure_e():
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    CE = 0
    if QE+QO+QO2+QS == 0:
        busy_time += (current_time - start_time)
    
    elif QE+QO+QO2+QS > 0:
        if QE > 0:
            QE -= 1
            CE = 1
            service_time = float(np.random.uniform(2,4,1))
            FEL.append((current_time + service_time, 'De'))
            FEL.sort()
            
        elif QS > 0:
            QS = 0
            CS = 1
            FEL.append((current_time + remaining_time, CS_event_type))
            FEL.sort()
            
        elif QO > 0:
            QO -= 1
            CO = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do'))
            FEL.sort()
            
        else:
            Qo2 -= 1
            CO2 = 1
            service_time = float(np.random.uniform(0,4,1))
            FEL.append((current_time + service_time, 'Do2'))
            FEL.sort()
    
    return None    

In [54]:
def departure(event_type):
    global CE, CO, CO2, CS, QE, QO, QO2, QS
    global FEL, current_time, remaining_time, CS_event_type, n, busy_time, start_time
    if (event_type == 'Do') | (event_type == 'Do2'):
        departure_o(event_type)
    elif event_type == 'De':
        departure_e()
    return None

In [55]:
while n < n_sim_calls:
    current_time, event_type = FEL.pop(0)
    if list(event_type)[0] == 'D':
        n += 1
        departure(event_type)
        pass
    elif list(event_type)[0] == 'A':
        arrival(event_type)
        pass
    print((current_time, event_type, n))
        

(0, 'Ae', 0)
(0, 'Ao', 0)
(3.089525077433633, 'De', 1)
(14.965537410579818, 'Ae', 1)
(17.407620336716082, 'De', 2)
(25.08720582333494, 'Ae', 2)
(27.42484060520263, 'De', 3)
(35.63917327655554, 'Ae', 3)
(38.24082644748514, 'De', 4)
(47.47053901223779, 'Ae', 4)
(51.40610303365939, 'De', 5)
(62.59502478829705, 'Ae', 5)
(64.83672880832955, 'De', 6)
(72.9516761616794, 'Ae', 6)
(76.06674404448545, 'De', 7)
(86.63330979880323, 'Ae', 7)
(89.62888763979788, 'De', 8)
(102.51677278368825, 'Ae', 8)
(104.77272563557868, 'De', 9)
(116.28469018093253, 'Ae', 9)
(119.3809560493847, 'De', 10)
(134.492845044759, 'Ae', 10)
(138.4715003294232, 'De', 11)
(153.18163208294004, 'Ae', 11)
(155.96188417600595, 'De', 12)
(166.98097803906614, 'Ae', 12)
(170.44712743385193, 'De', 13)
(179.36262377249753, 'Ae', 13)
(182.39223356384767, 'De', 14)
(195.1412206097062, 'Ae', 14)
(197.54040072998862, 'De', 15)
(207.02481195324427, 'Ae', 15)
(210.49047386725456, 'De', 16)
(222.94999178105607, 'Ae', 16)
(225.3615882673065,

In [56]:
print('=========Report========\n','busy time percentage', busy_time*100/current_time)

=========Report========
 busy time percentage 20.12732685411725
